In [12]:
# get a sample conversation

path_qs = '../data/cast19/human_questions.txt'

gs_qs = {}
with open(path_qs, 'r') as f:
    next(f)
    for line in f:
        q_id, q = line.split('\t')
#         print(q_id)
        gs_qs[q_id] = (q.strip('\n'))
# print(len(gs_qs))
# print(gs_qs['1_11'])

# collect all paragraphs
path = '../data/cast19/test_paragraphs.tsv'
qp_ids = {}
paras = {}
with open(path, encoding='utf-8') as f:
    for i, line in enumerate(f):
        q_id, para_id, para_text = line.rstrip().split('\t')
        if q_id not in qp_ids:
            qp_ids[q_id] = []
        qp_ids[q_id].append(para_id)
        if para_id not in paras:
            paras[para_id] = para_text
# print(len(paras))
# print(qp_ids)

# group question ids into sequences within the same conversation
from collections import defaultdict

question_sequences = defaultdict(list)
for q_id in qp_ids:
    conversation_id = q_id.split('_')[0]
    question_sequences[conversation_id].append(q_id)
#     print(gs_qs[q_id])
sample_conversation = question_sequences['31']
sample_qs = [gs_qs[i] for i in sample_conversation]
print(sample_qs)
print(len(sample_qs))
sample_ps_sets = [qp_ids[i] for i in sample_conversation]
# print(len(sample_ps_sets))
sample_ps = [p for i in sample_conversation for p in qp_ids[i]]
print(len(sample_ps))
sample_ps = set(sample_ps)
print(len(sample_ps))

['What is throat cancer?', 'Is throat cancer treatable?', 'Tell me about lung cancer.', 'What are the symptoms of lung cancer?', 'Can lung cancer spread to the throat?', 'What causes throat cancer?', 'What is the first sign of throat cancer?', 'Is throat cancer the same as esophageal cancer?', "What's the difference in symptoms of throat cancer and esophageal cancer?"]
9
737
605


In [2]:
for i, p_set in enumerate(sample_ps_sets):
    print(sample_qs[i], len(p_set))

What is throat cancer? 89
Is throat cancer treatable? 77
Tell me about lung cancer. 171
What are the symptoms of lung cancer? 98
Can lung cancer spread to the throat? 58
What causes throat cancer? 63
What is the first sign of throat cancer? 58
Is throat cancer the same as esophageal cancer? 68
What's the difference in symptoms of throat cancer and esophageal cancer? 55


In [14]:
# find duplicate words
from collections import Counter
import spacy

nlp = spacy.load('en_core_web_sm')

keywords = Counter()
for q in sample_qs:
    # preprocess question
    q = nlp(q.lower())
    keywords.update([token.text for token in q if not token.is_punct and not token.is_stop])
fts = [word for (word, freq) in keywords.items() if freq > 1]
print(fts)

['throat', 'cancer', 'lung', 'symptoms', 'esophageal']


In [16]:
# find duplicate paragraphs
pars = Counter()
for p_set in sample_ps_sets:
    pars.update(p_set)
fps = [word for (word, freq) in pars.items() if freq > 1]
print(len(fps))

112


In [18]:
# show relations between questions
for i, q in enumerate(sample_qs):
    # preprocess question
    terms = [token.text for token in nlp(q.lower()) if token.text in fts]
    dup_ps = [p for p in sample_ps_sets[i] if p in fps]
    print(q, terms, len(dup_ps))

What is throat cancer? ['throat', 'cancer'] 40
Is throat cancer treatable? ['throat', 'cancer'] 18
Tell me about lung cancer. ['lung', 'cancer'] 27
What are the symptoms of lung cancer? ['symptoms', 'lung', 'cancer'] 16
Can lung cancer spread to the throat? ['lung', 'cancer', 'throat'] 22
What causes throat cancer? ['throat', 'cancer'] 30
What is the first sign of throat cancer? ['throat', 'cancer'] 34
Is throat cancer the same as esophageal cancer? ['throat', 'cancer', 'esophageal', 'cancer'] 23
What's the difference in symptoms of throat cancer and esophageal cancer? ['symptoms', 'throat', 'cancer', 'esophageal', 'cancer'] 34
